[Click here ](https://wandb.ai/sxs200126/homework6A_DL/reports/Shishir_hw4_DL--VmlldzoxMzA3OTIz?accessToken=9xoffajdtcs8nilyrm00ukff9pgjzwddv3mgsj4mrpiql24u0k61wf9jgzuovig5) for complete report on Wandb

In [1]:
%%capture
!pip install wandb --upgrade

In [2]:
# Import wandb
import wandb

# Login to W&B
wandb.login()

wandb: Currently logged in as: sxs200126 (use `wandb login --relogin` to force relogin)


True

In [3]:
pip install torch-lr-finder

In [4]:
# Importing the necessary libraries
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchsummary import summary
import albumentations as A
import tensorflow as tf


from torch.optim.lr_scheduler import ReduceLROnPlateau, ExponentialLR, CyclicLR, OneCycleLR, StepLR, CosineAnnealingLR
#from torch_lr_finder import LRFinder

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import random

from datetime import datetime
from pathlib import Path
import plotly.io as pio
pio.renderers.default = 'colab'
from torchvision import models

In [5]:
# Import random function
import random

# Fix seed value
SEED = 2345
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
data_folder = Path('/content/drive/MyDrive/Data/DL')

In [8]:
folder = Path('/content/drive/MyDrive/Deep_Learning/HW6')

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

## **Data Download and Transform to tensor**

In [9]:
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
 #   transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Normalize the test set same as training set without augmentation
transform_test = transforms.Compose([
    transforms.Resize((224, 224)),                                     
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

In [10]:
# Transform to convert images to pytorch tensors and normalize the data

train_full = torchvision.datasets.CIFAR10(root=data_folder,
                                              train=True, 
                                              transform=transform_train,
                                              download=True)
trainset, validset = torch.utils.data.random_split(train_full, [40000, 10000], generator=torch.Generator().manual_seed(42) )
testset  = torchvision.datasets.CIFAR10(root=data_folder,
                                              train=False, 
                                              transform=transform_test,
                                              download=True)

Files already downloaded and verified
Files already downloaded and verified


In [11]:
classes = train_full.classes
classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [12]:
#class_count = {}
#for _, index in train_full:
#    label = classes[index]
#    if label not in class_count:
#        class_count[label] = 0
#    class_count[label] += 1
#class_count

In [13]:
train_full.data.shape

(50000, 32, 32, 3)

In [14]:
testset.data.shape

(10000, 32, 32, 3)

### Check Transformation

- Check if transformation are working correctly.
- The transformations are applied at the time of calling dataloader

In [15]:
check_loader = torch.utils.data.DataLoader(trainset, batch_size = 32, shuffle = True)

In [16]:
# check number of batches
len(check_loader)

1250

In [17]:
# check total training examples
len(check_loader.dataset)

40000

In [18]:
# check imputs and outputs 
for input, target in check_loader:
  print(f'shape of inputs is :{input.shape}')
  print(f'\nmax input value  :{input.max()}')
  print(f'\nmin input value  :{input.min()}')
  print(f'\nmean input value  :{input.mean()}')
  print(f'\nstd input value  :{input.std()}')
  print(f'\nshape of targets is :{target.shape}')
   
  break

shape of inputs is :torch.Size([32, 3, 224, 224])

max input value  :2.7537312507629395

min input value  :-2.429065704345703

mean input value  :-0.06401462107896805

std input value  :1.2636897563934326

shape of targets is :torch.Size([32])


First let's define labels for our dataset as dataset contains numerical values for now.

In [19]:
def get_CIFAR10_labels(labels):  
    """ 
    Function to generate labels.
    Input: numerical labels
    Output: actual string labels
    """

    # Create a list of labels
    text_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                   'dog', 'frog', 'horse', 'ship', 'truck']

    # Return text_labels according to numerical values
    return [text_labels[int(i)] for i in labels]

## **Small Dataset**

In [20]:
# n sample points
train_sample_size = int(len(trainset)/10)
valid_sample_size = int(len(validset)/10)

# Getting n random indices
train_subset_indices = random.sample(range(0, len(trainset)), train_sample_size)
valid_subset_indices = random.sample(range(0, len(testset)), valid_sample_size)

# Getting subset of dataset
train_subset = torch.utils.data.Subset(trainset, train_subset_indices)
valid_subset = torch.utils.data.Subset(validset, valid_subset_indices)

In [42]:
# load model with random weights
model = models.resnet101(pretrained=True)

In [43]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# Training Functions

## Training Epoch 

In [23]:
def train(train_loader, model, optimizer, loss_function, log_batch, log_interval, grad_clipping, max_norm):

  """ 
  Function for training the model in each epoch
  Input: iterator for train dataset, initial weights and bias, epochs, learning rate.
  Output: final weights, bias, train loss, train accuracy
  """
  # initilalize variables as global
  # these counts will be updated every epoch
  global example_ct_train
  global batch_ct_train

  # Training Loop loop
  # Initialize train_loss at the he start of the epoch
  running_train_loss = 0
  running_train_correct = 0
  
  # put the model in training mode
  model.train()

  # Iterate on batches from the dataset using train_loader
  for input, targets in train_loader:
    
    # move inputs and outputs to GPUs
    input = input.to(device)
    targets = targets.to(device)

    # Forward pass
    output = model(input)
    loss = loss_function(output, targets)

    # Correct prediction
    y_pred = torch.argmax(output, dim = 1)
    correct = torch.sum(y_pred == targets)

    example_ct_train +=  len(targets)
    batch_ct_train += 1

    # set gradients to zero 
    optimizer.zero_grad()

    # Backward pass
    loss.backward()

    # Gradient Clipping
    if grad_clipping:
      nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm, norm_type=2)

    # Update parameters using their gradient
    optimizer.step()

    #scheduler.step()
          
    # Add train loss of a batch 
    running_train_loss += loss.item()

    # Add Corect counts of a batch
    running_train_correct += correct

    # log batch loss and accuracy
    if log_batch:
      if ((batch_ct_train + 1) % log_interval) == 0:
        wandb.log({f"Train Batch Loss  :": loss})
        wandb.log({f"Train Batch Acc :": correct/len(targets)})
        #print(f'Learning rate: {scheduler.get_last_lr()}')

    
    
  # Calculate mean train loss for the whole dataset for a particular epoch
  train_loss = running_train_loss/len(train_loader)



  # Calculate accuracy for the whole dataset for a particular epoch
  train_acc = running_train_correct/len(train_loader.dataset)

  return train_loss, train_acc

## Validation/Test Epoch

In [24]:
def valid(loader, model, optimizer, loss_function, log_batch, log_interval):

  """ 
  Function for training the model and plotting the graph for train & valid loss vs epoch.
  Input: iterator for train dataset, initial weights and bias, epochs, learning rate, batch size.
  Output: final weights, bias and train loss and valid loss for each epoch.
  """

  # initilalize variables as global
  # these counts will be updated every epoch
  global example_ct_valid
  global batch_ct_valid

  # Validation loop
  # Initialize train_loss at the he strat of the epoch
  running_valid_loss = 0
  running_valid_correct = 0
  
  # put the model in evaluation mode
  model.eval()

  with torch.no_grad():
    for input,targets in loader:

      # move inputs and outputs to GPUs
      input = input.to(device)
      targets = targets.to(device)

      # Forward pass
      output = model(input)
      loss = loss_function(output,targets)

      # Correct Predictions
      y_pred = torch.argmax(output, dim = 1)
      correct = torch.sum(y_pred == targets)

      # count of images and batches
      example_ct_valid +=  len(targets)
      batch_ct_valid += 1

      # Add valid loss of a batch 
      running_valid_loss += loss.item()

      # Add correct count for each batch
      running_valid_correct += correct

      # log batch loss and accuracy
      if log_batch:
        if ((batch_ct_valid + 1) % log_interval) == 0:
          wandb.log({f"Valid Batch Loss  :": loss})
          wandb.log({f"Valid Batch Accuracy :": correct/len(targets)})


    # Calculate mean valid loss for the whole dataset for a particular epoch
    valid_loss = running_valid_loss/len(valid_loader)

    # scheduler step
    # scheduler.step(valid_loss)
    # scheduler.step()

    # Calculate accuracy for the whole dataset for a particular epoch
    valid_acc = running_valid_correct/len(valid_loader.dataset)
    
  return valid_loss, valid_acc

##  Model Training Loop


In [25]:
def train_loop(train_loader, valid_loader, model, loss_function, optimizer, epochs, device, patience, early_stopping,
               file_model):

  '''
  model: specify your model for training
  criterion: loss function 
  optimizer: optimizer like SGD , ADAM etc.
  train loader: function to carete batches for training data
  loader : function to create batches for valid data set
  file_model : specify file name for saving your model. This way we can upload the model weights from file. We will not to run model again.
  

  '''
  # Create lists to store train and valid loss at each epoch

  train_loss_history = []
  valid_loss_history = []
  train_acc_history = []
  valid_acc_history = []
  delta = 0
  best_score = None
  valid_loss_min = np.Inf
  counter_early_stop=0
  early_stop=False


  # Iterate for the given number of epochs
  for epoch in range(epochs):
    t0 = datetime.now()
    # Get train loss and accuracy for one epoch

    train_loss, train_acc = train(train_loader, model, optimizer, loss_function, 
                                  wandb.config.log_batch, wandb.config.log_interval,
                                  wandb.config.grad_clipping, wandb.config.max_norm)
    valid_loss, valid_acc = valid(valid_loader, model, optimizer, loss_function,
                                    wandb.config.log_batch, wandb.config.log_interval)

    dt = datetime.now() - t0

    # Save history of the Losses and accuracy
    train_loss_history.append(train_loss)
    train_acc_history.append(train_acc)
    valid_loss_history.append(valid_loss)
    valid_acc_history.append(valid_acc)

    if early_stopping:
      score = -valid_loss
      if best_score is None:
        best_score=score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving Model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss

      elif score < best_score + delta:
        counter_early_stop += 1
        print(f'Early stoping counter: {counter_early_stop} out of {patience}')
        if counter_early_stop > patience:
          early_stop = True

      
      else:
        best_score = score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), file_model)
        counter_early_stop=0
        valid_loss_min = valid_loss

      if early_stop:
        print('Early Stopping')
        break

    else:

      score = -valid_loss
      if best_score is None:
        best_score=score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving Model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss

      elif score < best_score + delta:
        print(f'Validation loss has not decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Not Saving Model...')
      
      else:
        best_score = score
        print(f'Validation loss has decreased ({valid_loss_min:.6f} --> {valid_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), file_model)
        valid_loss_min = valid_loss



    # Log the train and valid loss to W&B
    wandb.log({f"Train epoch Loss :": train_loss, f"Valid epoch Loss :": valid_loss })
    wandb.log({f"Train epoch Acc :": train_acc, f"Valid epoch Acc :": valid_acc})



    # Print the train loss and accuracy for given number of epochs, batch size and number of samples
    print(f'Epoch : {epoch+1} / {epochs}')
    print(f'Time to complete {epoch+1} is {dt}')
    # print(f'Learning rate: {scheduler.get_last_lr()}')
    # print(f'Learning rate: {scheduler._last_lr[0]}')
    print(f'Train Loss: {train_loss : .4f} | Train Accuracy: {train_acc * 100 : .4f}%')
    print(f'Valid Loss: {valid_loss : .4f} | Valid Accuracy: {valid_acc * 100 : .4f}%')
    print()
    torch.cuda.empty_cache()

  return train_loss_history, train_acc_history, valid_loss_history, valid_acc_history


# Model Training

In [136]:
hyperparameters = dict(
    epochs = 10,
    output_dim = 10, 
    
    batch_size = 128,
    learning_rate = 0.001,
    dataset="CIFAR10",
    architecture="resnet101_task4",
    log_interval = 100,
    log_batch = True,
    file_model = folder/'homework6A_task4_full_dataset.pt',
    grad_clipping = False,
    max_norm = 1,
    patience = 5,
    early_stopping = False,
    weight_decay = 0,
    scheduler_factor = 0.5,
    scheduler_patience = 0,
   )

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [137]:
device

device(type='cuda', index=0)

In [138]:
wandb.init(name = 'Shishir_hw6A_task4_full_dataset', project = 'homework6A_DL', config = hyperparameters)

Train Batch Acc :,▁▆▅▆▇█
Train Batch Loss :,█▄▃▃▂▁
Train epoch Acc :,▁▅▆▇▇▇▇▇████████████
Train epoch Loss :,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Valid Batch Accuracy :,▁
Valid Batch Loss :,▁
Valid epoch Acc :,▁▅▆▆▇▇▇▇████████████
Valid epoch Loss :,█▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Batch Acc :,0.98438
Train Batch Loss :,0.08688
Train epoch Acc :,0.98425


In [139]:
wandb.config.device = device
print(wandb.config.device )

cuda:0


## Specify Dataloader, Loss_function, Model, Optimizer, Weight Initialization

In [140]:
# Fix seed value
SEED = 2345
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Data Loader
train_loader = torch.utils.data.DataLoader(trainset, batch_size=wandb.config.batch_size, shuffle = True)
valid_loader = torch.utils.data.DataLoader(validset, batch_size=wandb.config.batch_size, shuffle = False)
test_loader = torch.utils.data.DataLoader(testset, batch_size=wandb.config.batch_size,   shuffle = False)



# cross entropy loss function
loss_function = nn.CrossEntropyLoss()

# Model 
# Task 1
#vgg16_task1 = models.vgg16(pretrained=True)
#vgg16_task1.classifier[6].out_features = wandb.config.output_dim

# freeze convolution weights
#for param in vgg16_task1.features.parameters():
#    param.requires_grad = False

#model = vgg16_task1 

# Task 2
#vgg16_task2 = models.vgg16(pretrained=True)
#vgg16_task2.classifier[6].out_features = wandb.config.output_dim

# freeze convolution weights
#for param in vgg16_task2.features[:24].parameters():
#    param.requires_grad = False

#model = vgg16_task2 

# Task 3
#vgg16_task3 = models.vgg16(pretrained=True)
#vgg16_task3.classifier[6].out_features = wandb.config.output_dim

# freeze convolution weights
#for param in vgg16_task3.features[:17].parameters():
#    param.requires_grad = False
#model = vgg16_task3 

# task 4

#resnet50_task4 = models.resnet50(pretrained=True)
#resnet50_task4.fc.out_features = wandb.config.output_dim

# Freeze all but last CNN block
#count = 0
#for child in resnet50_task4.children():
#  count+=1
#  if count < 8:
#    for param in child.parameters():
#        param.requires_grad = False

# task 4

resnet101_task4 = models.resnet101(pretrained=True)
resnet101_task4.fc.out_features = wandb.config.output_dim

# Freeze all but last CNN block
count = 0
for child in resnet101_task4.children():
  count+=1
  if count < 8:
    for param in child.parameters():
        param.requires_grad = False

model = resnet101_task4


def init_weights(m):
  if type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight)
        torch.nn.init.zeros_(m.bias)

  if type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight)
        torch.nn.init.zeros_(m.bias)

        
# apply initialization recursively  to all modules
# model.apply(init_weights)

#wandb.config.init_weights = init_weights

# put model to GPUs
model.to(wandb.config.device)

# Intialize stochiastic gradient descent optimizer
optimizer = torch.optim.SGD(model.parameters(), lr = wandb.config.learning_rate, weight_decay=wandb.config.weight_decay, momentum = 0.9)
#optimizer = torch.optim.Adam(model.parameters(), lr = wandb.config.learning_rate, weight_decay=wandb.config.weight_decay)

wandb.config.optimizer = optimizer

#scheduler = ReduceLROnPlateau(optimizer, mode='min', factor= wandb.config.scheduler_factor, 
#                              patience=wandb.config.scheduler_patience, verbose=True)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 1000, eta_min=0, last_epoch=- 1, verbose=False)
#
#scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.2, total_steps=len(train_loader) * 20 , 
#                                                epochs=5, three_phase=True, pct_start = 0.3)

#scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.2, total_steps=len(train_loader) * 30 , 
#                                                epochs=30, three_phase=True, pct_start = 0.2)

#scheduler = StepLR(optimizer, gamma=0.4,step_size=1, verbose=True)

## Sanity Check
- Check the loss without any training. For Cross entropy the expected value will be log(number of classes)

In [141]:
# Fix seed value

SEED = 2345
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

for input, targets in train_loader:
  
  # move inputs and outputs to GPUs
  input = input.to(device)
  targets = targets.to(device)
  model.eval()
  # Forward pass
  output = model(input)
  loss = loss_function(output, targets)
  print(f'Actual loss: {loss}')
  break

print(f'Expected Theoretical loss: {np.log(10)}')



Actual loss: 13.400022506713867
Expected Theoretical loss: 2.302585092994046


## Train Model and Save best model

In [142]:
wandb.watch(model, log = 'all', log_freq=25, log_graph=True)

wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [143]:
example_ct_train, batch_ct_train, example_ct_valid, batch_ct_valid = 0, 0, 0, 0
train_loss_history, train_acc_history, valid_loss_history, valid_acc_history = train_loop(train_loader, valid_loader, model, loss_function, optimizer, 
                                                                                          wandb.config.epochs, wandb.config.device,
                                                                                          wandb.config.patience, wandb.config.early_stopping,
                                                                                          wandb.config.file_model)

Validation loss has decreased (inf --> 0.388173). Saving Model...
Epoch : 1 / 10
Time to complete 1 is 0:06:58.169668
Train Loss:  0.9697 | Train Accuracy:  75.7800%
Valid Loss:  0.3882 | Valid Accuracy:  87.1600%

Validation loss has decreased (0.388173 --> 0.323358). Saving model...
Epoch : 2 / 10
Time to complete 2 is 0:06:59.309863
Train Loss:  0.3188 | Train Accuracy:  89.0825%
Valid Loss:  0.3234 | Valid Accuracy:  88.5900%

Validation loss has decreased (0.323358 --> 0.294334). Saving model...
Epoch : 3 / 10
Time to complete 3 is 0:06:59.354520
Train Loss:  0.2462 | Train Accuracy:  91.6375%
Valid Loss:  0.2943 | Valid Accuracy:  89.9000%

Validation loss has decreased (0.294334 --> 0.282134). Saving model...
Epoch : 4 / 10
Time to complete 4 is 0:06:58.752490
Train Loss:  0.1951 | Train Accuracy:  93.7400%
Valid Loss:  0.2821 | Valid Accuracy:  90.4300%

Validation loss has decreased (0.282134 --> 0.277442). Saving model...
Epoch : 5 / 10
Time to complete 5 is 0:06:59.599553
Tr

## Add Visulaization

### Add training images

In [144]:
# Get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

# Create grid of images
img_grid = torchvision.utils.make_grid(images[0:50], nrow = 5)

# Logging to W&B
images = wandb.Image(img_grid, caption = "Sample images")
wandb.log({"examples": images})

In [145]:
# Plot the train loss and test loss per iteration
fig = plt.figure(0)
plt.plot(train_loss_history, label = 'train loss')
plt.plot(valid_loss_history, label = 'valid loss')
plt.legend()

# Log the plot to W&B
wandb.log({"train-test loss per epoch": fig})

/usr/local/lib/python3.7/dist-packages/plotly/matplotlylib/renderer.py:410: UserWarning:

Bummer! Plotly can currently only draw Line2D objects from matplotlib that are in 'data' coordinates!

/usr/local/lib/python3.7/dist-packages/plotly/matplotlylib/renderer.py:512: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.



# **Accuracy and Predictions**

Now we have final values for weights and bias after training the model. We will use these values to make predictions on the test dataset.

## Function to get predictions

In [146]:
def get_acc_pred(data_loader, model):
  """ 
  Function to get predictions for a given test set and calculate accuracy.
  Input: Iterator to the test set.
  Output: Prections and Accuracy for test set.
  """
  model.eval()
  with torch.no_grad():
    # Array to store predicted labels
    predictions = torch.Tensor()
    predictions = predictions.to(device)

    # Array to store actual labels
    y = torch.Tensor()
    y = y.to(device)
    # Iterate over batches from test set
    for input, targets in data_loader:
      
      # move inputs and outputs to GPUs
      input = input.to(device)
      targets = targets.to(device)

      # Calculated the predicted labels
      output = model(input)

      # Choose the label with maximum probability
      indices = torch.argmax(output, dim = 1)

      # Add the predicted labels to the array
      predictions = torch.cat((predictions, indices)) 

      # Add the actual labels to the array
      y = torch.cat((y, targets)) 

    # Check for complete dataset if actual and predicted labels are same or not
    # Calculate accuracy
    acc = (predictions == y).float().mean()

  # Return array containing predictions and accuracy
  return predictions, acc
  

## Load saved model from file 

In [147]:
model_nn =  resnet101_task4
model_nn.to(device)
model_nn.load_state_dict(torch.load(wandb.config.file_model))

<All keys matched successfully>

In [148]:
print(wandb.config.file_model)

/content/drive/MyDrive/Deep_Learning/HW6/homework6A_task4_full_dataset.pt


In [149]:
# Get the prediction and accuracy for the test dataset
predictions, acc_test = get_acc_pred(test_loader, model_nn)

In [150]:
# Print Accuracy for test dataset
print(acc_test * 100)
wandb.config.test_accuracy = acc_test

tensor(91.2800, device='cuda:0')


**We have obtained 91.28 % accuracy on test dataset.**


In [41]:
#wandb.finish()